In [112]:
import geopandas as gd
import pandas as pd
import networkx as nx

In [113]:
path_geo_barrios = '../../data/geodata_bh/BAIRRO_POPULAR_4326.geojson'
path_geo_sectores='../../data/BH_regions.geojson'
path_geo_estado='../../data/data2/MG.json'
path_steps='../../data/data2/processed/steps_2.2.csv'

In [114]:
geo_barrios = gd.read_file(path_geo_barrios, driver='GeoJSON')
geo_sectores = gd.read_file(path_geo_sectores, driver='GeoJSON')
geo_estado = gd.read_file(path_geo_estado, driver='GeoJSON')
geo_barrios.NOME=geo_barrios.NOME.apply(lambda x: x+'_BH')
geo_sectores.NOME=geo_sectores.NOME.apply(lambda x: x+'_BH')
df=pd.read_csv(path_steps,encoding='latin-1' )
gdf_deliveries = gd.GeoDataFrame(df, geometry=gd.points_from_xy(df["lng"], df["lat"]))

### Obteniendo grafos de Localidades externas con barrios de BH

In [115]:
def get_tuples(ids_unique,group_id):
    tuples=[]
    for i in ids_unique:
        delivery=group_id.get_group(i).iloc[0]
        p_nome=''
        d_nome=''
        if delivery['action']=='pickup':
            p_nome=delivery['NOME']
            delivery_2=group_id.get_group(i).iloc[1]
            d_nome=delivery_2['NOME']
        else:
            d_nome=delivery['NOME']
            delivery_2=group_id.get_group(i).iloc[1]
            p_nome=delivery_2['NOME']

    tuples.append((p_nome,d_nome))
    return tuples

In [116]:
dfgeo_ext_BH = gd.sjoin(gdf_deliveries, geo_barrios, how='left', op='within')

/home/caloja/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_60827/3787890695.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  dfgeo_ext_BH = gd.sjoin(gdf_deliveries, geo_barrios, how='left', op='within')


In [117]:
df_geo_not_in_belohorizonte=dfgeo_ext_BH[dfgeo_ext_BH['NOME'].isnull()].drop(columns=['index_right','ID', 'CODIGO', 'NOME', 'AREA_KM2', 'PERIMETR_M'])
df_geo_in_belohorizonte=dfgeo_ext_BH[~dfgeo_ext_BH['NOME'].isnull()].drop(columns=['index_right','ID', 'CODIGO', 'AREA_KM2', 'PERIMETR_M'])

In [118]:
gdf_deliveries_mg_out = gd.sjoin(df_geo_not_in_belohorizonte, geo_estado, how='left', op='within')

/home/caloja/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_60827/3235581401.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  gdf_deliveries_mg_out = gd.sjoin(df_geo_not_in_belohorizonte, geo_estado, how='left', op='within')


In [119]:
gdf_deliveries_mg_out_si=gdf_deliveries_mg_out[~gdf_deliveries_mg_out.NOME.isnull()].drop(columns=['index_right','GEOCODIGO', 'UF'])
gdf_deliveries_mg_out_no=gdf_deliveries_mg_out[gdf_deliveries_mg_out.NOME.isnull()]

In [120]:
df_complete_ext_BH=pd.concat([df_geo_in_belohorizonte,gdf_deliveries_mg_out_si])

In [121]:
df_complete_ext_BH=df_complete_ext_BH[~df_complete_ext_BH.delivery_id.isin(gdf_deliveries_mg_out_no.delivery_id)]

In [122]:
group_id=df_complete_ext_BH.groupby('delivery_id')
ids_unique=df_complete_ext_BH.delivery_id.unique()
names=df_complete_ext_BH.NOME.unique()

In [123]:
G=nx.DiGraph()
G.add_nodes_from(names)

In [124]:
tuples=get_tuples(ids_unique,group_id)

In [125]:
G.add_edges_from(tuples)

In [126]:
paht_to_save="../../data/data2/processed/steps_graph_barrios.gexf"
nx.write_gexf(G, paht_to_save)

### Obteniendo grafos de Localidades externas con SECTORES de BH

In [127]:
dfgeo_ext__SECT_BH = gd.sjoin(gdf_deliveries, geo_estado, how='left', op='within')

/home/caloja/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_60827/1315085384.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  dfgeo_ext__SECT_BH = gd.sjoin(gdf_deliveries, geo_estado, how='left', op='within')


In [128]:
df_geo_not_in_belohorizonte_SECT=dfgeo_ext__SECT_BH[dfgeo_ext__SECT_BH['NOME'].isnull()].drop(columns=['index_right','GEOCODIGO','UF','NOME'])
df_geo_in_belohorizonte_SECT=dfgeo_ext__SECT_BH[~dfgeo_ext__SECT_BH['NOME'].isnull()].drop(columns=['index_right','GEOCODIGO','UF'])

In [129]:
gdf_deliveries_mg_out_SECT = gd.sjoin(df_geo_not_in_belohorizonte_SECT, geo_estado, how='left', op='within')

/home/caloja/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_60827/227671126.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  gdf_deliveries_mg_out_SECT = gd.sjoin(df_geo_not_in_belohorizonte_SECT, geo_estado, how='left', op='within')


In [130]:
gdf_deliveries_mg_out_si_SECT=gdf_deliveries_mg_out_SECT[~gdf_deliveries_mg_out_SECT.NOME.isnull()].drop(columns=['index_right','GEOCODIGO', 'UF'])
gdf_deliveries_mg_out_no_SECT=gdf_deliveries_mg_out_SECT[gdf_deliveries_mg_out_SECT.NOME.isnull()]

In [131]:
df_complete_ext_BH_SECT=pd.concat([df_geo_in_belohorizonte_SECT,gdf_deliveries_mg_out_si_SECT])

In [132]:
df_complete_ext_BH_SECT=df_complete_ext_BH_SECT[~df_complete_ext_BH_SECT.delivery_id.isin(gdf_deliveries_mg_out_no_SECT.delivery_id)]

In [133]:
group_id=df_complete_ext_BH_SECT.groupby('delivery_id')
ids_unique=df_complete_ext_BH_SECT.delivery_id.unique()
names=df_complete_ext_BH_SECT.NOME.unique()

In [134]:
G=nx.DiGraph()
G.add_nodes_from(names)

In [135]:
tuples=get_tuples(ids_unique,group_id)

In [136]:
G.add_edges_from(tuples)

In [137]:
paht_to_save="../../data/data2/processed/steps_graph_SECT.gexf"
nx.write_gexf(G, paht_to_save)

### Obteniendo grafo de barrios en BH

In [138]:
dfgeo_ext__BARR_BH = gd.sjoin(gdf_deliveries, geo_barrios, how='left', op='within')

/home/caloja/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_60827/3964527023.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  dfgeo_ext__BARR_BH = gd.sjoin(gdf_deliveries, geo_barrios, how='left', op='within')


In [139]:
dfgeo_ext__BARR_BH=dfgeo_ext__BARR_BH.drop(columns=['index_right','ID', 'CODIGO', 'AREA_KM2', 'PERIMETR_M'])

In [140]:
df_geo_not_in_belohorizonte_BARR=dfgeo_ext__BARR_BH[dfgeo_ext__BARR_BH['NOME'].isnull()]
df_geo_in_belohorizonte_BARR=dfgeo_ext__BARR_BH[~dfgeo_ext__BARR_BH['NOME'].isnull()]

In [141]:
df_complete_ext_BH_BARR=df_geo_in_belohorizonte_BARR[~df_geo_in_belohorizonte_BARR.delivery_id.isin(df_geo_not_in_belohorizonte_BARR.delivery_id)]

In [ ]:
group_id=df_complete_ext_BH_BARR.groupby('delivery_id')
ids_unique=df_complete_ext_BH_BARR.delivery_id.unique()
names=df_complete_ext_BH_BARR.NOME.unique()

In [ ]:
G=nx.DiGraph()
G.add_nodes_from(names)

In [ ]:
tuples=get_tuples(ids_unique,group_id)

In [ ]:
G.add_edges_from(tuples)

In [ ]:
paht_to_save="../../data/data2/processed/steps_graph_BARR.gexf"
nx.write_gexf(G, paht_to_save)